# Connect to the printer

In [ ]:
import peripage

printer = peripage.Printer('45:54:07:0d:39:48', peripage.PrinterType.A6p)
await printer.connect()
await printer.reset()

# Get information about printer

In [ ]:
print(f'Name: {await printer.getDeviceName()}')
print(f'S/N: {await printer.getDeviceSerialNumber()}')
print(f'F/W: {await printer.getDeviceFirmware()}')
print(f'Battery: {await printer.getDeviceBattery()}%')
print(f'H/W: {await printer.getDeviceHardware()}')
print(f'MAC: {await printer.getDeviceMAC()}')
print(f'Full: {await printer.getDeviceFull()}')

# Sample print

In [ ]:
await printer.writeASCII('Hello World?\n')
await printer.printBreak(100)

# Print random image

In [ ]:
import numpy
from PIL import Image
imarray = numpy.random.rand(printer.getRowWidth(),printer.getRowWidth(), 3) * 255
im = Image.fromarray(imarray.astype('uint8')).convert('L')

# Set print concentration
await printer.setConcentration(1)

# Print image & break
await printer.printImage(im)
await printer.printBreak(100)

# Infinite print using generator

In [ ]:
# Ping-pong slider
def slider():
    direct = True
    pos = 0
    while True:
        s = '00' * pos + 'ff' + '00' * (printer.getRowBytes() - pos - 1)
        if direct:
            pos = pos + 1
            if pos == printer.getRowBytes() - 1:
                direct = False
        else:
            pos = pos - 1
            if pos == 0:
                direct = True
        yield bytes.fromhex(s)

# Infinite printing, slow
printer.printRowBytesIterator(slider(), 0.25)

# Print using generator in limited page length

In [ ]:
# Ping-pong slider
def slider():
    direct = True
    pos = 0
    while True:
        s = '00' * pos + 'ff' + '00' * (printer.getRowBytes() - pos - 1)
        if direct:
            pos = pos + 1
            if pos == printer.getRowBytes() - 1:
                direct = False
        else:
            pos = pos - 1
            if pos == 0:
                direct = True
        yield bytes.fromhex(s)

# Print on page 200 px length
printer.printRowBytesIteratorOfSize(slider(), 200, 0.01)
printer.printBreak(100)

In [ ]:
import random

# Random byte rows
def rand():
    while True:
        s = ''.join([random.choice('0123456789abcdef') for n in range(printer.getRowWidth())])
        yield s

# Print on page 200 px length
await printer.printRowBytesIteratorOfSize(rand(), 200, 0.01)
await printer.printBreak(100)